# 집값 예측

- Data fields
    - ID : 집을 구분하는 번호
    - date : 집을 구매한 날짜
    - price : 집의 가격(Target variable)
    - bedrooms : 침실의 수
    - bathrooms : 침실 개수 당 화장실의 수(화장실의 수 / 침실의 수 )
    - sqft_living : 주거 공간의 평방 피트(면적)
    - sqft_lot : 부지의 평방 피트(면적)
    - floors : 집의 층 수
    - waterfront : 집의 전방에 강이 흐르는지 유무 (a.k.a. 리버뷰)
    - view : 집이 얼마나 좋아 보이는지의 정도
    - condition : 집의 전반적인 상태   #건물 연식이 몇년이 지나면 가격 같음
    - grade : King County grading 시스템 기준으로 매긴 집의 등급   
    - sqft_above : 지하실을 제외한 평방 피트(면적)
    - sqft_basement : 지하실의 평방 피트(면적)   # 지하실 있는 데이터는 따로 취급?
    - yr_built : 지어진 년도
    - yr_renovated : 집을 재건축한 년도
    - zipcode : 우편번호
    - lat : 위도
    - long : 경도
    - sqft_living15 : 2015년 기준 주거 공간의 평방 피트(면적, 집을 재건축했다면, 변화가 있을 수 있음)
    - sqft_lot15 : 2015년 기준 부지의 평방 피트(면적, 집을 재건축했다면, 변화가 있을 수 있음)

In [5]:
import pandas as pd
import numpy as np
import random

In [14]:
home = pd.read_csv("dataset/train.csv")

In [13]:
# train 셋을 추출해서 home으로 만들어줍니다.
home.head()  

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,0,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,1,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
2,2,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503
3,3,20140627T000000,257500.0,3,2.25,1715,6819,2.0,0,0,...,7,1715,0,1995,0,98003,47.3097,-122.327,2238,6819
4,4,20150115T000000,291850.0,3,1.50,1060,9711,1.0,0,0,...,7,1060,0,1963,0,98198,47.4095,-122.315,1650,9711


In [17]:
# 이중 1000개의 데이터를 임의로 샘플 추출 합니다.
rd_idx = [random.randint(0,15034) for _ in range(1000)]   # 0 ~ 15034 개까지의 인데스 중 랜덤 샘플 1000개 추출
sam1000 = home.iloc[rd_idx].reset_index(drop=True)  # 샘플 추출
sam1000

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,12078,20141112T000000,179950.0,3,1.00,920,4875,1.0,0,0,...,6,920,0,1969,0,98042,47.3648,-122.081,960,5200
1,1208,20150327T000000,700000.0,3,1.75,2000,14733,1.0,0,0,...,8,2000,0,1958,0,98005,47.6001,-122.178,2620,14733
2,898,20141023T000000,780000.0,4,2.50,3690,13609,2.0,0,2,...,10,3690,0,1996,0,98075,47.5872,-121.972,3600,13609
3,842,20141020T000000,5110800.0,5,5.25,8010,45517,2.0,1,4,...,12,5990,2020,1999,0,98033,47.6767,-122.211,3430,26788
4,14906,20140709T000000,440000.0,3,1.50,1270,1413,3.0,0,0,...,8,1270,0,2007,0,98115,47.6989,-122.317,1270,1413
5,11619,20140609T000000,585000.0,4,1.75,2470,131790,2.0,0,2,...,8,2470,0,1937,0,98070,47.4421,-122.444,1470,92747
6,14358,20140725T000000,674000.0,3,2.50,1920,3624,2.0,0,0,...,9,1920,0,2003,0,98052,47.6647,-122.087,1930,3533
7,10761,20150403T000000,336750.0,4,2.25,1720,7803,1.0,0,0,...,8,1350,370,1955,0,98146,47.5119,-122.337,1720,7803
8,12728,20140923T000000,1230000.0,5,4.00,4390,6656,2.0,0,0,...,9,2930,1460,2008,0,98199,47.6480,-122.397,1560,6656
9,9995,20150318T000000,353500.0,4,2.50,3250,4650,2.0,0,0,...,8,3250,0,2007,0,98031,47.4004,-122.203,2960,4650
